<img src="Imatges/PortadaData.jpg" width="1200" />

## Data
Though we live in times of data hype and terabytes of data accumulate endlessly, data is often kept at very different places, in different formats. Fortunately, there are official governmental sources of data which are reliable. These sources are not completely coherent in terms of format, but they are manageable.
In this project I’m going to use three main sources of data:
* Data from the Cartographic Institute of Catalonia [ICC](https://www.icgc.cat/Administracio-i-empresa/Descarregues/Capes-de-geoinformacio/Seccions-censals)
* Data from Girona’s local council that can be obtained either from Girona Open Data [GOD](https://www.girona.cat/opendata/) or from l’[Observatori](https://terra.girona.cat/apps/observatori/).
* Data from [Foursquare](https://foursquare.com/)
### Geographic data <img src="Imatges/DataProcessing_small.jpg"  style="float:right" />
The data can be downloaded directly from the Jupiter notebook, or can be downloaded externally and kept in a local file to be further imported in the notebook. I chose the second system to avoid that network problems interfere with the development process.
In the figure beside, there is a schematic representation of the transformations that have been performed in the Geografic data from ICC and GOD. In the original sources, geographical data is kept in Shapefile format which is a common format used in geographical information systems (GIS). However, the geojson format is increasingly used in data science and is the format that is used to feed map tools like Folium and Choropleth. Thus, the first step will be to transform .shp files into .json files thanks to a library PyShp. I will define the function *shpToGeoJSON* were I will use the [PyShp](https://pypi.org/project/pyshp/) tools to read .shp files and return the information in the form of python dictionaries.
In both ICC and GOD the geographical data is kept in utm coordinates, but many current applications are better fed with WGS84 coordinates. So, I will transform the utm coordinates to WGS84 coordinates with the help of a library [PyProj](https://pypi.org/project/pyproject/). For this job I’ll create the function *getWGS84Coordinates*.
Once the data is in json format and the coordinates are coded with the WGS84 system, I’ll make some small transformations like changing some town codes by the official name of the town, this is for the sake of readability of data when observing them. I’ll also create a merged code joining the district code and the section code to uniquely identify each census section in Girona.
### Foursquare data
Foursquare is a powerful database which that can be easily accessed with APIs provided in its developer page. However, there are some limitations that forces us to do some extra work. The main limitation I found is that the results of searches for venues are given in circle area defined by a radius. I want to assign each venue to the census section to where it belongs to. In order to achieve that I will make the following steps:
 1.	Determine the centroid of each census are which are, for the most, quite irregular polygons. For this I use the library *scipy.spatial* which has a tool to calculate the convex hull of a polygon. Then the centroid of the convex hull is the mean of the x coordinates and the mean of the y coordinates of the hull. I used this approximation because it gives better results than the simpler approximation of determining the centroid as the means of the points of the raw polygon (data not shown).
 2.	Once the centroid is found I use the function *distanceBetweenPoints* to determine the longest distance between the centroid and all the vertices. This will be the distance used to assign the parameter RADIUS in the Foursquare search.
 3.	Next step is to retrieve Foursquare data using latitude and longitude of the centroid of each census section and the radius as the longest distance between centroid and vertices. But this will yield a lot of overlapping information as the circle defined by the radius in each census section will intersect with the circles of the neighboring sections. 
 4.	Once venue data are obtained, venues that do not belong to the census section must be filtered out. In order to do this, I’ll have to check if venue coordinates are inside the polygon which defines the census section. This can be easily done using the function *polygon.contains* of the [Shapely](https://pypi.org/project/Shapely/) library that I’ll execute inside the local function *ifInDistrict*.
 5.	The final step is to convert all data in a Panda’s data frame.

 ### Statistical data
There are many open sources of interesting and reliable statistical information which is organized at different levels like [Eurostat](https://ec.europa.eu/eurostat) at the european level, [IDESCAT](https://www.idescat.cat/?lang=en) at national level, and the aforementioned GOD and l'Observatori at local level in Girona. All this data can be downloaded in csv format and easily transformed in Panda's data frames.
 

